SYSTEME DE RECOMMANDATION DE FILMS

Nous avons utilisé un dataset MovieLens 100k Small Dataset provenant de Kaggle contenant 2 fichiers: 
    - movies.csv contenant les informations sur les films(movieId, title et genre)
    - rating.csv contenant les évaluations des utilisateurs sur les films(userId, movieId, rating, timestamp)

Le fichier movies.csv contient 3 variables et 9125 observations; le fichier rating.csv a 4 variables dont 100004 observations.

1- IMPORT ET CHARGEMENT DES DONNEES

In [1]:
import pandas as pd

movies = pd.read_csv("data/movies.csv", sep=";")
ratings = pd.read_csv("data/rating.csv", sep=";")

movies.head()
# ratings.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


2- DATA CLEANINIG MINIMAL

a- Structure des données

In [2]:
movies.shape

(9125, 3)

In [3]:
ratings.shape

(100004, 4)

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9125 non-null   int64 
 1   title    9125 non-null   object
 2   genres   9125 non-null   object
dtypes: int64(1), object(2)
memory usage: 214.0+ KB


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


b- Vérification des valeurs manquantes

In [6]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [7]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

c- Suppression des doublons

In [8]:
movies = movies.drop_duplicates(subset=["movieId"])

In [9]:
ratings = ratings.drop_duplicates(subset=["userId", "movieId"])

d- Nettoyage de la colonne genres

In [10]:
movies['genres'] = movies['genres'].str.replace('|', ' ', regex=False)

e- Suppression des films sans genre

In [11]:
movies = movies[movies['genres'] != '(no genres listed)']

cette action est effectuée parce qu'un film sans genre est impossible à comparer aux autres.

In [12]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


3- Implémentation du coeur du système de recommandation

a- Cas 1: Content-Based: recommandation basée uniquement sur les genres

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer #librairie pour transformer les textes en nombres
from sklearn.metrics.pairwise import cosine_similarity #librairie pour calculer la similarité entre deux vecteurs(choses)

#on crée le transformeur TF-IDF qui sait comment transformer des mots en nombres, pour le moment il n'a encore rien appris
tfidf = TfidfVectorizer()

#on fait apprendre le transformeur, il va apprendre tous les genres de films(fit) puis les tranformer en nombres(transform) 
tfidf_matrix = tfidf.fit_transform(movies['genres'])

#on calcule la similarité entre les films
cosine_sim = cosine_similarity(tfidf_matrix)

#on crée une fonction qui prend en entrée le titre d'un film et qui retourne une liste de 10 films recommandés
def recommend_movies(movie_title, cosine_sim=cosine_sim):

    #on vérifie si le film existe
    if movie_title not in movies['title'].values:
        return "Film non trouvé"
    
    #on récupère l'index du film donné en entrée
    idx = movies[movies['title'] == movie_title].index[0]
    
    #on récupère les similarités de ce film avec tous les autres films
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #on trie les films en fonction de leur similarité avec le film donné en entrée
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #on prend les 10 films les plus similaires (en excluant le premier qui est le film lui-même)
    sim_scores = sim_scores[1:11]
    
    #on récupère les indices des films recommandés
    movie_indices = [i[0] for i in sim_scores]
    
    #on retourne les titres des films recommandés
    return movies.iloc[movie_indices][['title', 'genres']]

recommend_movies("Mr. Holland's Opus (1995)")

,title,genres
25,Othello (1995),Drama
30,Dangerous Minds (1995),Drama
38,"Cry, the Beloved Country (1995)",Drama
41,Restoration (1995),Drama
52,Georgia (1995),Drama
53,Home for the Holidays (1995),Drama
58,Mr. Holland's Opus (1995),Drama
104,Margaret's Museum (1995),Drama
109,"Boys of St. Vincent, The (1992)",Drama
112,"Star Maker, The (Uomo delle stelle, L') (1995)",Drama


b- Cas 2: Collaborative-Based Filtering

Item-Based: on compare les films entre eux. on regarde les films qui sont notés de manière similaire. A et B aiment les memes films, B a aimé un film que A n'a pas encore vu alors on recommande A ce film. 

b1- On crée la matrice utilisateur-film

In [37]:
user_movie_matrix = ratings.pivot_table(
    index = "userId",
    columns = "movieId",
    values = "rating"
)

user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


b2- on remplace les films non notés par 0

In [38]:
user_movie_matrix = user_movie_matrix.fillna(0)

user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


b3- Similarité entre films(on compare les colonnes)

In [39]:
movie_similarity = cosine_similarity(user_movie_matrix.T)

b4- on crée un dataFrame de similarité

In [41]:
movie_similarity_df = pd.DataFrame(
    movie_similarity,
    index = user_movie_matrix.columns,
    columns = user_movie_matrix.columns
)

movie_similarity_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.394511,0.306516,0.133614,0.245102,0.377086,0.278629,0.063031,0.117499,0.310689,...,0.055829,0.031902,0.079755,0.079755,0.079755,0.079755,0.079755,0.0,0.0,0.055829
2,0.394511,1.000000,0.217492,0.164651,0.278476,0.222003,0.207299,0.223524,0.113669,0.418124,...,0.000000,0.055038,0.068797,0.082557,0.082557,0.137594,0.068797,0.0,0.0,0.000000
3,0.306516,0.217492,1.000000,0.177012,0.370732,0.247499,0.435648,0.127574,0.306717,0.191255,...,0.000000,0.000000,0.000000,0.116226,0.116226,0.000000,0.000000,0.0,0.0,0.000000
4,0.133614,0.164651,0.177012,1.000000,0.179556,0.072518,0.184626,0.501513,0.254630,0.111447,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
5,0.245102,0.278476,0.370732,0.179556,1.000000,0.272645,0.388476,0.194113,0.367941,0.246846,...,0.000000,0.176845,0.000000,0.117897,0.117897,0.000000,0.000000,0.0,0.0,0.000000


b5- Fonction de recommandation item-based

In [50]:
def recommend_movies_collaborative(movie_title, n=10):

    if movie_title not in movies['title'].values:
        return None

    movie_id = movies[movies['title'] == movie_title]['movieId'].values[0]

    sim_scores = movie_similarity_df[movie_id].sort_values(ascending=False)
    sim_scores = sim_scores.iloc[1:n+1]

    return movies[movies['movieId'].isin(sim_scores.index)][['title', 'genres']]

recommend_movies_collaborative("Mr. Holland's Opus (1995)")

,title,genres
16,Sense and Sensibility (1995),Drama Romance
34,Dead Man Walking (1995),Crime Drama
122,"Birdcage, The (1996)",Comedy
561,Mission: Impossible (1996),Action Adventure Mystery Thriller
599,"Truth About Cats & Dogs, The (1996)",Comedy Romance
615,"Rock, The (1996)",Action Adventure Thriller
644,Independence Day (a.k.a. ID4) (1996),Action Adventure Sci-Fi Thriller
658,Phenomenon (1996),Drama Romance
661,"Time to Kill, A (1996)",Drama Thriller
866,Willy Wonka & the Chocolate Factory (1971),Children Comedy Fantasy Musical
